# low-rank Krylov Subspace Approximation (KSA) for density matrix SCF optimization

In [3]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
from seqm.seqm_functions.read_xyz import read_xyz

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
%%time
DTYPE = torch.float64
torch.set_default_dtype(DTYPE)
files = [
        'C380.xyz',
        ]

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device, dtype=DTYPE)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))

# parapeters dict for KSA-XL-BOMD needs to be passed to 'scf_converger'
xl_bomd_params={'k':6, 'max_rank':3, 'err_threshold':0.0, 'T_el':1500}

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [3,xl_bomd_params], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                                         # [3, xl_bomd_params], low-rank KSA scf optimization
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)

### Create electronic structure driver:
esdriver = Electronic_Structure(seqm_parameters).to(device)

### Run esdriver on molecules:
esdriver(molecules)


/home/maxim/anaconda3/envs/pyseqm_4/lib/python3.9/site-packages/PYSEQM-0.0.1-py3.9.egg/seqm/seqm_functions/diag.py:134: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at /opt/conda/conda-bld/pytorch_1670525551200/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2794.)
  e0,v = torch.symeig(x0,eigenvectors=True,upper=True)
